### Creating a prediction map from a trained and saved 1D CNN model which is based on 88 features (5 spectral bands, 43 features (VIs), 40 texture features and predicting tiller density of an image (pixel based prediction map)

In [1]:
# Import libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rasterio
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model

# Function to calculate different (43) vegetation indices

def calculate_ndvi(nir_band, red_band):
    ndvi = (nir_band - red_band) / (nir_band + red_band)
    return ndvi

def calculate_gndvi(nir_band, green_band):
    gndvi = (nir_band - green_band) / (nir_band + green_band)
    return gndvi

def calculate_rvi_1(nir_band, red_band):
    rvi_1 = nir_band / red_band
    return rvi_1

def calculate_gci(nir_band, green_band):
    gci = (nir_band / green_band) - 1.0
    return gci

def calculate_rgvi(red_band, green_band):
    rgvi = red_band / green_band
    return rgvi

def calculate_dvi(nir_band, red_band):
    dvi = nir_band - red_band
    return dvi

def calculate_savi(nir_band, red_band, L=0.5):
    savi = ((nir_band - red_band) / (nir_band + red_band + L)) * (1.0 + L)
    return savi

def calculate_msavi(nir_band, red_band):
    msavi = 0.5 * ((2.0 * nir_band) + 1.0 - np.sqrt(np.square(2.0 * nir_band + 1.0) - 8.0 * (nir_band - red_band)))
    return msavi

def calculate_osavi(nir_band, red_band):
    osavi = (nir_band - red_band) / (nir_band + red_band + 0.16)
    return osavi

def calculate_rdvi(nir_band, red_band):
    rdvi = np.sqrt((np.square(nir_band - red_band)) / (nir_band + red_band))
    return rdvi

def calculate_tvi(nir_band, red_band, green_band):
    tvi = 60.0 * (nir_band - green_band) - 100.0 * (red_band - green_band)
    return tvi

def calculate_tsavi(nir_band, red_band, s=0.33, a=0.5, X=1.5):
    tsavi = (s * (nir_band - s * red_band - a)) / (a * nir_band + red_band - a * s + X * (1 + np.square(s)))
    return tsavi

def calculate_pvi(nir_band, red_band, a=0.3, b=0.5):
    pvi = (nir_band - a * red_band - b) / (np.sqrt(1 + np.square(a)))
    return pvi

def calculate_savi_2(nir_band, red_band, a=0.96916, b=0.084726):
    savi_2 = nir_band / (red_band + (b / a))
    return savi_2

def calculate_atsavi(nir_band, red_band, a=0.96916, b=0.084726, X=0.08):
    atsavi = (a * (-a * red_band - b)) / (a * nir_band + red_band - a * b + X * (1 + np.square(a)))
    return atsavi

def calculate_ndwi(nir_band, green_band):
    ndwi = (green_band - nir_band) / (green_band + nir_band)
    return ndwi

def calculate_ikaw(red_band, blue_band):
    ikaw = (red_band - blue_band) / (red_band + blue_band)
    return ikaw

def calculate_srpi(red_band, blue_band):
    srpi = blue_band / red_band
    return srpi

def calculate_rvi_2(nir_band, green_band):
    rvi_2 = nir_band / green_band
    return rvi_2

def calculate_mcari(red_edge_band, red_band, green_band):
    mcari = (red_edge_band - red_band - 0.2 * (red_edge_band - green_band)) * (red_edge_band / red_band)
    return mcari

def calculate_mcari_1(nir_band, red_band, green_band):
    mcari_1 = 1.2 * (2.5 * (nir_band - red_band) - 1.3 * (nir_band - green_band))
    return mcari_1

def calculate_mcari_2(nir_band, red_band, green_band):
    mcari_2 = 1.5 * (2.5 * (nir_band - red_band) - 1.3 * (nir_band - green_band)) * (np.square(2.0 * nir_band + 1)) - (6.0 * nir_band - 5.0 * red_band) - 0.5
    return mcari_2

def calculate_mtvi_1(nir_band, red_band, green_band):
    mtvi_1 = 1.2 * (1.2 * (nir_band - green_band) - 2.5 * (red_band - green_band))
    return mtvi_1

def calculate_mtvi_2(nir_band, green_band):
    mtvi_2 = (nir_band - green_band) / (nir_band + green_band)
    return mtvi_2

def calculate_r_mcari_mtvi2(nir_band, red_band, green_band):
    r_mcari_mtvi2 = 1.5 * (1.2 * (nir_band - green_band) - 2.5 * (red_band - green_band)) * (np.square(2 * nir_band + 1)) - (6.0 * nir_band - 5.0 * red_band) - 0.5
    return r_mcari_mtvi2

def calculate_evi(nir_band, red_band, blue_band):
    G = 2.5  # Gain factor
    C1 = 6.0  # Coefficient
    C2 = 7.5  # Coefficient
    L = 1.0   # Canopy background adjustment
    evi = G * ((nir_band - red_band) / (nir_band + C1 * red_band - C2 * blue_band + L))
    return evi

def calculate_datt(nir_band, red_edge_band, red_band):
    datt = (nir_band - red_edge_band) / (nir_band - red_band)
    return datt

def calculate_dnci(red_edge, green):
    dnci = (red_edge_band - green_band) / (red_edge_band + green_band)
    return dnci

def calculate_psri(red_edge_band, red_band, green_band):
    psri = (red_band - green_band) / red_edge_band
    return psri

def calculate_sipi(nir_band, red_band, blue_band):
    sipi = (nir_band - blue_band) / (nir_band + red_band)
    return sipi

def calculate_spvi(nir_band, red_band, green_band):
    spvi = 0.4 * 3.7 * (nir_band - red_band) - 1.2 * np.abs(green_band - red_band)
    return spvi

def calculate_tcari(red_edge, red, green):
    tcari = 3.0 * ((red_edge_band - red_band) - 0.2 * (red_edge_band - green_band) * (red_edge_band / red_band))
    return tcari

def calculate_r_tcari_osavi(nir_band, red_edge_band, red, green):
    r_tcari_osavi = (3.0 * ((red_edge_band - red_band) - 0.2 * (red_edge_band - green_band) * (red_edge_band / red_band))) / ((nir_band - red_band) / (nir_band + red_band + 0.16))
    return r_tcari_osavi

def calculate_reri(nir_band, red_edge_band, red_band):
    reri = (red_edge_band - red_band) / nir_band
    return reri

def calculate_ndre(nir_band, red_edge_band):
    ndre = (nir_band - red_edge_band) / (nir_band + red_edge_band)
    return ndre

def calculate_mtci(nir_band, red_edge_band, red_band):
    mtci = (nir_band - red_edge_band) / (red_edge_band - red_band)
    return mtci

def calculate_evi_2(nir_band, red_band):
    evi_2 = 2.5 * ((nir_band - red_band) / (nir_band + 2.4 * red_band + 1.0))
    return evi_2

def calculate_reci(nir_band, red_edge_band):
    reci = (nir_band / red_edge_band) - 1
    return reci

def calculate_nexg(red_band, green_band, blue_band):
    nexg = (2 * green_band - red_band - blue_band) / (green_band + red_band + blue_band)
    return nexg

def calculate_ngrdi(red_band, green_band):
    ngrdi = (green_band - red_band) / (green_band + red_band)
    return ngrdi

def calculate_endvi(nir_band, green_band, blue_band):
    endvi = (nir_band + green_band - 2.0 * blue_band) / (nir_band + green_band + 2.0 * blue_band)
    return endvi

def calculate_ari_2(nir_band, red_edge_band, green_band):
    ari_2 = nir_band * ((1.0 / green_band) - (1.0 / red_edge_band))
    return ari_2

def calculate_cri_2(red_edge_band, green_band):
    cri_2 = (1.0 / green_band) - (1.0 / red_edge_band)
    return cri_2

# Open the main .tif image (assuming it contains 5 spectral bands)
with rasterio.open(r'D:\resample_30cm_new\hayes_clip2_resampled_30cm.tif') as src:
    # Read the spectral bands from the image
    spectral_bands = src.read()  # Shape (bands, height, width)
    height, width = spectral_bands.shape[1], spectral_bands.shape[2]
    blue_band = spectral_bands[0]
    green_band = spectral_bands[1]
    red_band = spectral_bands[2]
    red_edge_band = spectral_bands[3]
    nir_band = spectral_bands[4]

# Calculate vegetation indices
ndvi = calculate_ndvi(nir_band, red_band)
gndvi = calculate_gndvi(nir_band, green_band)
rvi_1 = calculate_rvi_1(nir_band, red_band)
gci = calculate_gci(nir_band, green_band)
rgvi = calculate_rgvi(red_band, green_band)
dvi = calculate_dvi(nir_band, red_band)
savi = calculate_savi(nir_band, red_band)
msavi = calculate_msavi(nir_band, red_band)
osavi = calculate_osavi(nir_band, red_band)
rdvi = calculate_rdvi(nir_band, red_band)
tvi = calculate_tvi(nir_band, red_band, green_band)
tsavi = calculate_tsavi(nir_band, red_band)
pvi = calculate_pvi(nir_band, red_band)
savi_2 = calculate_savi_2(nir_band, red_band)
atsavi = calculate_atsavi(nir_band, red_band)
ndwi = calculate_ndwi(nir_band, green_band)
ikaw = calculate_ikaw(red_band, blue_band)
srpi = calculate_srpi(red_band, blue_band)
rvi_2 = calculate_rvi_2(nir_band, green_band)
mcari = calculate_mcari(red_edge_band, red_band, green_band)
mcari_1 = calculate_mcari_1(nir_band, red_band, green_band)
mcari_2 = calculate_mcari_2(nir_band, red_band, green_band)
mtvi_1 = calculate_mtvi_1(nir_band, red_band, green_band)
mtvi_2 = calculate_mtvi_2(nir_band, green_band)
r_mcari_mtvi2 = calculate_r_mcari_mtvi2(nir_band, red_band, green_band)
evi = calculate_evi(nir_band, red_band, blue_band)
datt = calculate_datt(nir_band, red_edge_band, red_band)
dnci = calculate_dnci(red_edge_band, green_band)
psri = calculate_psri(red_edge_band, red_band, green_band)
sipi = calculate_sipi(nir_band, red_band, blue_band)
spvi = calculate_spvi(nir_band, red_band, green_band)
tcari = calculate_tcari(red_edge_band, red_band, green_band)
r_tcari_osavi = calculate_r_tcari_osavi(nir_band, red_edge_band, red_band, green_band)
reri = calculate_reri(nir_band, red_edge_band, red_band)
ndre = calculate_ndre(nir_band, red_edge_band)
mtci = calculate_mtci(nir_band, red_edge_band, red_band)
evi_2 = calculate_evi_2(nir_band, red_band)
reci = calculate_reci(nir_band, red_edge_band)
nexg = calculate_nexg(red_band, green_band, blue_band)
ngrdi = calculate_ngrdi(red_band, green_band)
endvi = calculate_endvi(nir_band, green_band, blue_band)
ari_2 = calculate_ari_2(nir_band, red_edge_band, green_band)
cri_2 = calculate_cri_2(red_edge_band, green_band) 

# Stack all spectral bands and vegetation indices
# Example: Assuming you have 43 vegetation indices including NDVI
vegetation_indices = np.stack([ndvi,gndvi,rvi_1,gci,rgvi,dvi,savi,msavi,osavi,rdvi,tvi,tsavi,pvi,savi_2,atsavi,ndwi,ikaw,srpi,rvi_2,mcari,mcari_1,mcari_2,
               mtvi_1,mtvi_2,r_mcari_mtvi2,evi,datt,dnci,psri,sipi,spvi,tcari,r_tcari_osavi,reri,ndre,mtci,evi_2,reci,nexg,ngrdi,endvi,ari_2,cri_2], axis=0)
# Stack spectral bands and vegetation indices
spectral_vegetation_stack = np.concatenate([spectral_bands, vegetation_indices], axis=0)

# Open the separate .tif image for texture features (assuming it contains 40 texture bands)
with rasterio.open(r'D:\resample_30cm_new\hayes_clip2_tex3_final_resampled_30cm.tif') as texture_src:
    # Read the texture features from the image
    texture_features = texture_src.read()  # Shape (40, height, width)

# Ensure the texture features have the same spatial dimensions
if texture_features.shape[1] != height or texture_features.shape[2] != width:
    raise ValueError("Texture image dimensions do not match the spectral image dimensions.")
    
# Stack the spectral+vegetation indices with the texture features
total_features_stack = np.concatenate([spectral_vegetation_stack, texture_features], axis=0)

# Prepare the data for normalization and prediction
# Reshape the data to (num_pixels, num_features), where num_features = 88
# Flatten height and width to create a 2D array of pixels

num_features = total_features_stack.shape[0]
pixel_features = total_features_stack.reshape(num_features, height * width).T  # Shape (num_pixels, num_features)

# Normalize the features using StandardScaler
scaler = StandardScaler()
normalized_features = scaler.fit_transform(pixel_features)  # Shape (num_pixels, num_features)
print(normalized_features.shape)


## When there is no data value
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Check for NaN and infinite values
print("NaN values present before cleaning:", np.isnan(normalized_features).any())
print("Infinite values present before cleaning:", np.isinf(normalized_features).any())

# Step 1: Replace infinite values with NaN (if needed)
normalized_features[np.isinf(normalized_features)] = np.nan

# Step 2: Create an imputer to handle NaN values
imputer = SimpleImputer(strategy='mean')

# Step 3: Fit and transform to fill NaN values
normalized_features = imputer.fit_transform(normalized_features)

# Step 4: Check again for NaN or infinite values after imputation
print("NaN values present after cleaning:", np.isnan(normalized_features).any())
print("Infinite values present after cleaning:", np.isinf(normalized_features).any())

# Step 5: Scale the cleaned data
scaler = StandardScaler()
normalized_features = scaler.fit_transform(normalized_features)

print("Data scaling complete.")




# Predicting map from the model
# Define your custom R² metric function
def Keras_r2_score(y_true, y_pred):
    SS_res = K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return (1 - SS_res / (SS_tot + K.epsilon()))

# Load the trained 1D CNN model and pass the custom metric using custom_objects
model = load_model(
    r'D:\resample_30cm_new\crop_tiller_density_model_1dcnn.h5',
    custom_objects={'Keras_r2_score': Keras_r2_score}
)


# Predict the tiller density using the trained model
predicted_tiller_density = model.predict(normalized_features)  # Output shape: (num_pixels, 1)

# Reshape the predicted tiller_density back to the original image shape (height, width)
predicted_tiller_density_image = predicted_tiller_density.reshape(height, width)

# Save the predicted tiller density to a new .tif image
output_filename = r'D:\resample_30cm_new\predicted_tiller_density.tif'
with rasterio.open(
    output_filename,
    'w',
    driver='GTiff',
    height=height,
    width=width,
    count=1,
    dtype=predicted_tiller_density_image.dtype,
    crs=src.crs,
    transform=src.transform,
) as dst:
    dst.write(predicted_tiller_density_image, 1)

print(f"Predicted tiller density saved to {output_filename}")

# Plot the map/result
plt.figure(figsize=(10, 8))  # Set figure size if needed
plt.imshow(predicted_tiller_density_image, cmap='viridis')  # You can change the colormap if needed
plt.colorbar(label='Tiller Density')  # Add colorbar with label
plt.title("Tiller Density Map")  # Add title to the plot
plt.xlabel("Pixel X")  # Label for X-axis
plt.ylabel("Pixel Y")  # Label for Y-axis
plt.show()  # Display the plot


# Showing the histogram of predicted image
# Load the raster file
tiller_density_raster = r'D:\resample_30cm_new\predicted_tiller_density.tif'

# Open the raster and read the pixel values
with rasterio.open(tiller_density_raster) as src:
    tiller_density = src.read(1)  # Read the first band (assuming single-band raster)

# Flatten the 2D array of tiller density values to 1D array for histogram plotting
tiller_density_flat = tiller_density.flatten()

# Plot the histogram of tiller density values
plt.figure(figsize=(10, 6))
plt.hist(tiller_density_flat, bins=50, color='blue', edgecolor='black', alpha=0.7)
plt.title('Histogram of Predicted Tiller Density')
plt.xlabel('Tiller Density')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()


# Sving the tiller density pixel values
# working directory
os.chdir(r'D:\resample_30cm_new')
# Load the raster file
tiller_density_raster = r'D:\resample_30cm_new\predicted_tiller_density.tif'

# Open the raster and read the pixel values
with rasterio.open(tiller_density_raster) as src:
    tiller_density = src.read(1)  # Read the first band (assuming single-band raster)
    height, width = tiller_density.shape  # Get the height and width of the raster

# Flatten the 2D array of tiller density values to 1D array
tiller_density_flat = tiller_density.flatten()

# Prepare pixel data for saving, including Pixel_ID and Row_Col
pixel_data = []
pixel_id = 1
for row in range(height):
    for col in range(width):
        row_col = f"{row}_{col}"
        tiller_density_value = tiller_density[row, col]
        pixel_data.append([pixel_id, row_col, tiller_density_value])
        pixel_id += 1

# Convert pixel data to a DataFrame
df = pd.DataFrame(pixel_data, columns=['Pixel_ID', 'Row_Col', 'Tiller_Density'])

# Get the maximum number of rows Excel can handle
max_rows_per_sheet = 1048576  # Excel has a 1,048,576 row limit

# Calculate the number of sheets needed
total_rows = len(df)
num_sheets = (total_rows // max_rows_per_sheet) + 1

# Create an Excel writer to store all sheets in one file
output_file = 'tiller_density.xlsx'
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for i in range(num_sheets):
        start_row = i * max_rows_per_sheet
        end_row = min((i + 1) * max_rows_per_sheet, total_rows)
        
        # Slice the DataFrame for each sheet
        df_chunk = df.iloc[start_row:end_row]
        
        # Save the chunk to a new sheet
        sheet_name = f'Sheet_{i+1}'
        df_chunk.to_excel(writer, sheet_name=sheet_name, index=False)
        
        print(f'Saved {sheet_name} in {output_file}')

print(f'All data saved to {output_file}')

